Universidade Federal do Rio Grande do Sul (UFRGS)   
Programa de Pós-Graduação em Engenharia Civil (PPGEC)   

# PEC00025: Introduction to Vibration Theory


## Test P2 (2023/1): Discrete and continuous mdof systems

---

**NAME:** <br/>
**CARD:** 



In [2]:
# Importing Python modules required for this notebook
# (this cell must be executed with "shift+enter" before any other Python cell)

import numpy as np
import matplotlib.pyplot as plt
import pickle as pk
import scipy.linalg as sc

from MRPy import MRPy


In [3]:
def vibration_modes(K, M):

# Uses scipy to solve the standard eigenvalue problem
    w2, Phi = sc.eig(K, M)

# Ensure ascending order of eigenvalues
    iw  = w2.argsort()
    w2  = w2[iw]
    Phi = Phi[:,iw]

# Eigenvalues to vibration frequencies
    wk  = np.sqrt(np.real(w2)) 
    fk  = wk/2/np.pi

    return fk, wk, Phi


## Questão 1

O sistema estrutural abaixo, com 2 g.d.l., representa um pórtico plano dotado de um amortecedor de massa sintonizada. Calcule os coeficientes de rigidez $k_1$ e $k_2$, e as respectivas formas modais, correspondentes às frequências naturais dadas. 

<img src="resources/tests/PEC00025A_231_P2_Q1.png" alt="Question 1" width="540px"/>


### Solução

As matrizes de massa e de rigidez são:

$$ \mathbf{K} =  \left[ \begin{array}{cc}
                  k_1 + k_2 &  -k_2  \\
                      - k_2 &   k_2
                 \end{array} \right] $$
                 
$$ \mathbf{M} =  \left[ \begin{array}{cc}
                        M_1 &    0   \\
                         0  &   M_2
                 \end{array} \right] $$

Lembrando agora o problema de autovalores e autovetores:

$$ \mathbf{K} \, \vec{\varphi}_i = \omega_i^2 \, \mathbf{M} \, \vec{\varphi}_i  $$ 

Considerando que as formas modais são normalizadas pela sua coordenada em $M_1$:

$$\left[ \begin{array}{cc}
   k_1 + k_2 &  -k_2  \\
       - k_2 &   k_2
  \end{array} \right] \cdot
  \left[ \begin{array}{c}
       1      \\
   \varphi_i
  \end{array} \right] = \omega_i^2
  \left[ \begin{array}{cc}
   M_1 &    0   \\
    0  &   M_2
  \end{array} \right] \cdot
  \left[ \begin{array}{c}
       1      \\
   \varphi_i
  \end{array} \right] $$

Isso resulta em um par de equações:

$$\begin{align*}
(k_1 + k_2) - k_2 \varphi_i &= \omega_i^2 M_1 \\
      -k_2  + k_2 \varphi_i &= \omega_i^2 M_2 \varphi_i 
\end{align*}$$

para $i = 1$ e $i = 2$. Portanto, tem-se um sistema com 4 equações para 4 incógnitas
($k_1$, $k_2$, $\varphi_1$, $\varphi_2$). A dificuldade está na não-linearidade. 
Somando-se as duas equações:

$$ k_1 = \omega_i^2 (M_1 + M_2\varphi_i) $$

Isolando-se $k_2$ na primeira equação:

$$ k_2 = \frac{\omega_i^2 M_1 - k_1}{1 - \varphi_i} $$

Inicialmente vamos atribuir um valor inicial às rigidezes para se obter as formas modais.


In [33]:
M1   =  10000.
M2   =  500.

w1_2 = (2*np.pi*1.00)**2
w2_2 = (2*np.pi*1.00)**2

k1   =  w1_2*M1
k2   =  w2_2*M2

KG  = np.array([[ k1+k2,  -k2 ],[-k2,  k2]])
MG  = np.array([[   M1,    0  ],[ 0,   M2]])

fk, wk, Phi = vibration_modes(KG, MG)

phi1 = Phi[:,0]/Phi[0,0]    # normalizando pela coordenada da massa M1
phi2 = Phi[:,1]/Phi[0,1]

print(fk, '\n', phi1, phi2)


[0.89442719 1.11803399] 
 [1. 5.] [ 1. -4.]


Agora calculamos $k_1$ e $k_2$.


In [34]:
k1a1 =  (wk[0]**2)*(M1 + M2*phi1[1])
k2a1 = ((wk[0]**2)*M1 - k1a1)/(1 - phi1[1])

k1a2 =  (wk[1]**2)*(M1 + M2*phi2[1])
k2a2 = ((wk[1]**2)*M1 - k1a2)/(1 - phi2[1])

print('\t', k1a1, k2a1, '\t', k1a2, k2a2)


	 394784.1760435744 19739.2088021787 	 394784.1760435743 19739.208802178713


## Questão 2

Para a estrutura do problema anterior, calcule as máximas amplitudes de deslocamento de cada massa para uma carga dinâmica harmônica aplicada na massa $M_1$.

$$F(t) = F_0 \sin(2\pi f_0 t)$$

onde $F_0 = 1$kN e $f_0 = 1$Hz . 


### Solução



In [ ]:
# Simulação das forças NODAIS

N   = 8192
Td  = 64
F0  = 1000.
f0  = 1.00

F1  = MRPy(np.vstack((F1, np.zeros_like(F1))), fs=fs)

f2  = F1.plot_time(fig=2, figsize=(10,4), axis_t=[0, F1.Td, -200, 800]);


In [ ]:
# Cálculo das forças MODAIS

zk1 = np.array([0.01, 0.01])
Mk1 = np.diag(np.dot(Phi1.T, np.dot(M1, Phi1)))
Kk1 = Mk1*(wk1**2)
Fk1 = MRPy(np.dot(Phi1.T, F1), fs=F1.fs)

f3  = Fk1.plot_time(fig=3, figsize=(10,5), axis_t=[0, F1.Td, -500, 500])
 

In [ ]:
# Cálculo dos deslocamentos por superposição modal

# Mass division must be a matrix operation...
ak1 = MRPy(np.dot(np.diag(1/Mk1), Fk1), fs=Fk1.fs)

# ... and now solving
uk1 = ak1.sdof_Duhamel(fk1, zk1)             # modal space solution
uN1 = MRPy(np.dot(Phi1, uk1), fs=uk1.fs)     # back to nodal solution

# Resultado no domínio do tempo
f4  = uN1.plot_time(4, figsize=(10,5), axis_t=[0, uN1.Td, -0.08, 0.08])

# Resultado no domínio da frequência (para confirmar picos)
f5  = uN1.plot_freq(5, figsize=(10,5), axis_f=[0, 10, 0, 0.001])

up1 = uN1.max(axis=1)
print('Deslocamento de pico da massa 1 é {0:6.4f}m'.format(up1[0]))
print('Deslocamento de pico da massa 2 é {0:6.4f}m'.format(up1[1]))


#### Método 2: por condições iniciais (carga como impulso de Dirac)


In [ ]:
I0  = 2*F0*Td/3                         # impulse is ∫F(t)dt...
v   = I0/m                              # ... which is converted to a initial velocity

u0  = np.array([[0., 0.]]).T            # column vector with the initial displacements
v0  = np.array([[v,  0.]]).T            # column vector with the initial velocities

qMu = np.dot(np.dot(Phi1.T, M1), u0)
qMv = np.dot(np.dot(Phi1.T, M1), v0)

thk = np.zeros_like(Mk1)                # phase angles to be calculated
u0k = np.zeros_like(Mk1)                # modal response amplitude to be calculated

for k in range(2):

# If there are initial displacements only
#   thk[k] = -np.pi/2
#   u0k[k] =  qMu[k]/Mk1[k]/np.sin(thk[k])

# If there are initial velocities only
    thk[k] =  np.arctan(wk1[k]*qMu[k]/qMv[k])
    u0k[k] =  qMv[k]/Mk1[k]/np.cos(thk[k])/wk1[k]

    print('Mode {0} with phase {1:5.2f}rad and amplitude {2:7.4f}m'.format(k+1, thk[k], u0k[k]))


In [ ]:
# Build the modal responses as harmonic functions with given properties
uk  = MRPy.harmonic(NX=2, N=N, fs=F1.fs, X0=u0k, f0=fk1, phi=thk)

# Calculate the NODAL responses superposing all modal responses
uN = MRPy(np.dot(Phi1, uk), fs=F1.fs)

f6  = uN.plot_time(6, figsize=(10,5), axis_t=[0, uN.Td, -0.08, 0.08])

up  = uN.max(axis=1)
print('Deslocamento de pico da massa 1 é {0:6.4f}m'.format(up[0]))
print('Deslocamento de pico da massa 2 é {0:6.4f}m'.format(up[1]))


...


## Questão 3

Para a viga com as restrições de apoio dadas, proponha uma forma aproximada para o primeiro modo de vibração, $\varphi(x)$, e calcule a respectiva frequência natural em função do comprimento, $L$, da massa por unidade de comprimento, $\mu$, e da rigidez à flexão, $EI$.

<img src="resources/tests/PEC00025A_231_P2_Q3.png" alt="Question 3" width="480px"/>  


### Solução


In [ ]:
uA =  0.590      # deslocamento da extremidade superior esquerda (m)
uB =  uA         # deslocamento da extremidade superior direita (m)

θA = -0.0894     # rotação da extremidade superior esquerda (rad)
θB = -0.0887     # rotação da extremidade superior direita (rad)


Estes valores podem ser interpolados utilizando as funções de interpolação dadas em aula, para termos uma expressão analítica para a linha elástica, com a qual pode-se calcular a energia de deformação e a energia cinética de referência.


In [ ]:
# Dados do problema
L  = 4.           # comprimento das barras (m)
EI = 6500.        # rigidez à flexão (Nm2)
μ  = 20.          # massa por unidade de comprimento (kg/m)
F  = 1000.        # carga estática arbitrária aplicada (N)

# Discretização do comprimento das barras
x  = np.linspace(0, L, 200)
dx = L/200

# Lambda functions para interpolação dos deslocamentos
phi = []
phi.append(lambda xi:  1 - 3*xi*xi + 2*xi*xi*xi)
phi.append(lambda xi:  L*(xi - 2*xi*xi + xi*xi*xi))
phi.append(lambda xi:  3*xi*xi - 2*xi*xi*xi)
phi.append(lambda xi:  L*(-xi*xi + xi*xi*xi ))

# Lambda functions para interpolação das curvaturas
phixx = []
phixx.append(lambda xi: (-6 + 12*xi)/L/L) 
phixx.append(lambda xi: (-4 +  6*xi)/L  ) 
phixx.append(lambda xi: ( 6 - 12*xi)/L/L) 
phixx.append(lambda xi: (-2 +  6*xi)/L  ) 


In [ ]:
# Deslocamentos interpolados para as três barras

# coluna da esquerda (x de baixo pra cima)
w1 =  0*phi[0](x/L) +  0*phi[1](x/L) + uA*phi[2](x/L) - θA*phi[3](x/L)
# viga superior (x da esquerda para direita)
w2 =  0*phi[0](x/L) + θA*phi[1](x/L) +  0*phi[2](x/L) + θB*phi[3](x/L)
# coluna direita (x de baixo pra cima)
w3 =  0*phi[0](x/L) +  0*phi[1](x/L) + uB*phi[2](x/L) - θB*phi[3](x/L)

f7 = plt.figure(7, figsize=(5,5))
s  = 2   # escala das deformações

plt.plot(s*w1, x, 'b', s*uA + x, L + s*w2, 'b', L + s*w3, x, 'b')
plt.axis('equal')
plt.grid(True)


Esse conjunto de linhas deformadas será utilizado como forma modal para o cálculo da respectiva frequência natural
de vibração livre através do quociente de Rayleigh.

Observe que além dos deslocamentos transversais a cada barra, tem-se
também o deslocamento da viga para a direita, que representa a maior
parte da energia cinética do sistema!


In [ ]:
# Energia cinética de referência

# Deslocamento da viga para a direita!
Tv = μ*L*(uA**2)/2     

# Deslocamentos transversais das três barras
Tr = Tv + μ*(np.trapz(w1**2 + w2**2 + w3**2, dx=dx))/2

print('A energia cinética de referência é {0:4.2f} J/m.'.format(Tr))
print('A massa da viga para a direita representa {0:3.1f}%.'.format(100*Tv/Tr))


A energia potencial elástica pode ser calculada pelo trabalho da força externa:

In [ ]:
# Trabalho da força externa
V  = F*uA/2

print('A energia potencial elástica é {0:4.1f} J.'.format(V))


E finalmente o cálculo pelo quociente de Rayleigh:

In [ ]:
fn = np.sqrt(V/Tr)/(2*np.pi)

print('A frequência fundamental do pórtico é menor que {0:5.3f} Hz.'.format(fn))


O cálculo da energia potencial elástica também pode ser feito pela curvatura:

In [ ]:
# Curvaturas

w1xx =  0*phixx[0](x/L) +  0*phixx[1](x/L) + uA*phixx[2](x/L) - θA*phixx[3](x/L)
w2xx =  0*phixx[0](x/L) + θA*phixx[1](x/L) +  0*phixx[2](x/L) + θB*phixx[3](x/L)
w3xx =  0*phixx[0](x/L) +  0*phixx[1](x/L) + uB*phixx[2](x/L) - θB*phixx[3](x/L)

V    = EI*(np.trapz(w1xx**2 + w2xx**2 + w3xx**2, dx=dx))/2

print('A energia potencial elástica é {0:4.1f} J.'.format(V))


...


## Questão 4

Para a viga do problema anterior, com os valores dados abaixo, calcule a máxima amplitude de deslocamento para o peso próprio sendo aplicado de forma súbita a partir do tempo $t = 0$, ou seja, como uma função passo unitário, $h(t)$.

<img src="resources/tests/PEC00025A_231_P2_Q4.png" alt="Question 4" width="540px"/>  


Estime o valor r.m.s. e o valor de pico do deslocamento horizontal $u(t)$ na
extremidade esquerda da viga. Calcule a correspondente _força estática equivalente_.


### Solução

...


In [ ]:
# Cálculo das propriedades modais

Mk = 2*Tr
wk = 2*np.pi*fn
Kk = wk*wk*Mk
zk = 0.01

# Deslocamento estático a partir da rigidez modal
Fk = 1000*uA      # força modal
uk = Fk/Kk        # deslocamento estático modal
ue = uA*uk        # deslocamento nodal

print('Massa modal:           {0:5.1f} kg.'.format(Mk))
print('Rigidez modal:         {0:4.1f} N/m.'.format(Kk))
print('Deslocamento estático: {0:5.3f} m.'.format(ue))


A força (definida pelo espectro) é aplicada na extremidade
esquerda da viga, na horizontal, onde o deslocamento na forma
modal tem amplitude $u_A$. Portanto o espectro da força modal é:


In [ ]:
M  =  4097             # discretização do domínio da frequência
σF =  50               # valor r.m.s. da força
r  =  10.
f  =  np.linspace(0, 2*r, M)
fs =  2*f[-1]

SF =  np.zeros_like(f)

SF[f > 1/r] = (σF**2)/(2*np.log(r))*(1/f[f > 1/r])
SF[f < 1/r] =  0.
SF[f >  r ] =  0.

sF2 =  np.trapz(SF, f)
sF  =  np.sqrt(sF2)

print('Amplitude r.m.s. pela integral do espectro é {0:4.1f} N.\n'.format(sF))

SFk = uA*uA*SF      # espectro da força modal Fk(t)

plt.figure(3, figsize=(8,4), clear=True)
plt.semilogy(f, SFk);
plt.grid(True)
plt.axis([0, 1.2*r, 1e01, 1e04])
plt.xlabel('Frequência (Hz)')
plt.ylabel('Espectro da força modal [N²/Hz]');


Uma vez definido o espectro da força modal, podemos calcular a 
resposta modal no domínio da frequência.

O espectro do deslocamento NODAL (extremidade esquerda da viga)
é obtido multiplicando-se o espectro do deslocamento MODAL por 
$u_A^2$, da mesma forma como foi feito para a força modal.


In [ ]:
Hf2 =  lambda fi: 1/( (1 - (fi/fn)**2)**2 + (2*zk*(fi/fn))**2 )/(Kk**2)

SU  =  uA*uA*Hf2(f)*SFk

plt.figure(4, figsize=(8,4), clear=True)
plt.semilogy(f, SU);
plt.grid(True)
plt.axis([0, 1.2*r, 1e-10, 1e02])
plt.xlabel('Frequência (Hz)')
plt.ylabel('Espectro do deslocamento [m²/Hz]');


Finalmente, faz-se a análise estatística da resposta em deslocamento
a partir do espectro.


In [ ]:
sU2 =  np.trapz(    SU, f)
sU4 =  np.trapz(f*f*SU, f)

nu  =  np.sqrt(sU4/sU2)
lnu =  np.sqrt(2*np.log(60*nu))  # Tempo de excitação é 60 segundos!
g   =  lnu + 0.5772/lnu

sU  =  np.sqrt(sU2)
up  =  g*sU

print('Fator de pico da resposta em deslocamento é    {0:6.2f}.'.format(g))
print('Amplitude r.m.s. da resposta em deslocamento é {0:4.0f}mm.'.format(1000*sU))
print('Valor de pico da resposta em deslocamento é    {0:4.0f}mm.'.format(1000*up))


Podemos usar a rigidez aparente da análise do Ftool para 
calcular a força estática equivalente. 


In [ ]:
k   = 1000/uA
Feq = up*k

print('Força estática equivalente é {0:4.1f}N.'.format(Feq))
